<a href="https://colab.research.google.com/github/limshaocong/ReqBERT/blob/main/Notebooks/t1_finetuning_seqclass_frnfr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries

The following libraries are required for this notebook. If run locally, it can be commented out.


* `transformers` - an excellent library by Hugging Face that provides a high-level abstraction for fine-tuning of transformer-based models, and a model hub of pre-trained models
* `datasets` - a dataset library by Hugging Face. All datasets used for this thesis are uploaded to this dataset hub. 
* `torch` - PyTorch. When running the `allenai/scibert_scivocab_cased` pre-trained model with this notebook on GCP Vertex AI, `torch` needs to be installed.
* `seqeval` - a library for evaluation of sequence labelling tasks

In [ ]:
! pip install --user datasets transformers torch

While Google Colab provides free GPU access, the specific GPU (K80/ T4/ P100) available is a function of the subscription level (Free Tier/ Colab Pro/ Colab Pro+) and usage trends. Alternatively, Google Cloud Platform provides US$300 in free credits that can be used for paid GPU access.

In [ ]:
! nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-87d97a92-1e04-722d-6b69-f215eb87255c)


# Import & Pre-process Data

These dictionaries will minimize the amount of adjustment within the remainder of the script.

In [ ]:
# This dict allows the correct tokenizer to be imported
# ReqBERT, ReqRoBERTa and ReqSciBERT uses the tokenizers from the
# corresponding base models
model_type_dict = {
    'bert-base-cased' : 'bert-base-cased',
    'roberta-base' : 'roberta-base',
    'allenai/scibert_scivocab_cased' : 'allenai/scibert_scivocab_cased',
    'limsc/reqbert-tapt-epoch29' : 'bert-base-cased',
    'limsc/reqroberta-tapt-epoch43' : 'roberta-base',
    'limsc/reqscibert-tapt-epoch20' : 'allenai/scibert_scivocab_cased',
}

model_name_dict = {
    'bert-base-cased' : 'bert',
    'roberta-base' : 'roberta',
    'allenai/scibert_scivocab_cased' : 'scibert',
    'limsc/reqbert-tapt-epoch29' : 'reqbert-e29',
    'limsc/reqroberta-tapt-epoch43' : 'reqroberta-e43',
    'limsc/reqscibert-tapt-epoch20' : 'reqscibert-e20'
}

task_name_dict = {
    'limsc/fr-nfr-classification' : 'frnfr',
    'limsc/subclass-classification' : 'subclass',
    'limsc/requirements-entity-recognition' : 'er'
}

The dataset for the [Task 1 - FR/NFR Classification](https://huggingface.co/datasets/limsc/subclass-classification) is uploaded onto the HuggingFace hub.

In [ ]:
from datasets import load_dataset

ds_name = 'limsc/fr-nfr-classification'
ds = load_dataset(ds_name)
ds

Downloading:   0%|          | 0.00/923 [00:00<?, ?B/s]

Using custom data configuration limsc--fr-nfr-classification-665313997f6e1426


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/limsc___parquet/limsc--fr-nfr-classification-665313997f6e1426/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['reqs', 'is_functional'],
        num_rows: 144
    })
    val: Dataset({
        features: ['reqs', 'is_functional'],
        num_rows: 143
    })
    train: Dataset({
        features: ['reqs', 'is_functional'],
        num_rows: 669
    })
})

To transform natural language requirements into a BERT-compatible format, the text must first be tokenized. This is performed using a pre-trained tokenizer.

Select any of the six LMs and import the corresponding tokenizer.

* `bert-base-cased`
* `roberta-base`
* `allenai/scibert_scivocab_cased`
* `limsc/reqbert-tapt-epoch29`
* `limsc/reqroberta-tapt-epoch43`
* `limsc/reqscibert-tapt-epoch20`

In [ ]:
model_checkpoint = 'limsc/reqroberta-tapt-epoch43'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_type_dict[model_checkpoint],
    use_fast = True
)

def encode(requirements):
    return tokenizer(requirements['reqs'], truncation = True, max_length = 128)

tokenized_ds = ds.map(encode, batched = True)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_ds

DatasetDict({
    test: Dataset({
        features: ['reqs', 'is_functional', 'input_ids', 'attention_mask'],
        num_rows: 144
    })
    val: Dataset({
        features: ['reqs', 'is_functional', 'input_ids', 'attention_mask'],
        num_rows: 143
    })
    train: Dataset({
        features: ['reqs', 'is_functional', 'input_ids', 'attention_mask'],
        num_rows: 669
    })
})

# Model Fine-tuning (Single Loop)

Pre-batch datasets in to batches of specified size.


In [ ]:
from transformers import DataCollatorWithPadding

batch_size = 32
output_col = 'is_functional'

data_collator = DataCollatorWithPadding(
    tokenizer = tokenizer,
    return_tensors = 'tf'
)

def batching(tokenized_ds, batch_size):

  batched_train_ds = tokenized_ds['train'].to_tf_dataset(
      columns = ['attention_mask', 'input_ids', 'token_type_ids'],
      label_cols = [output_col],
      shuffle = False,
      drop_remainder = False,
      collate_fn = data_collator,
      batch_size = batch_size
  )

  batched_val_ds = tokenized_ds['val'].to_tf_dataset(
      columns = ['attention_mask', 'input_ids', 'token_type_ids'],
      label_cols = [output_col],
      shuffle = False,
      drop_remainder = False,
      collate_fn = data_collator,
      batch_size = batch_size
  )

  batched_test_ds = tokenized_ds['test'].to_tf_dataset(
      columns = ['attention_mask', 'input_ids', 'token_type_ids'],
      label_cols = [output_col],
      shuffle = False,
      drop_remainder = False,
      collate_fn = data_collator,
      batch_size = batch_size
  )

  return batched_train_ds, batched_val_ds, batched_test_ds

batched_train_ds, batched_val_ds, batched_test_ds = batching(tokenized_ds, batch_size)

For tf2.6 onwards, the weights of the classification head is only affected by seeds set using tf.random_set_seed. See discussion on [stackoverflow](https://stackoverflow.com/questions/32419510/how-to-get-reproducible-results-in-keras).

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, create_optimizer

seed = 67897
tf.random.set_seed(seed)

num_epochs = 3
initial_lr = 2e-5

def create_model(num_epochs, initial_lr):

  from_pt = True if model_checkpoint == 'allenai/scibert_scivocab_cased' else False

  model = TFAutoModelForSequenceClassification.from_pretrained(
      model_checkpoint,
      num_labels = 2,
      from_pt = from_pt
  )

  batches_per_epoch = len(tokenized_ds['train']) // batch_size
  total_train_steps = int(batches_per_epoch * num_epochs)

  optimizer, schedule = create_optimizer(
      init_lr = initial_lr,
      num_warmup_steps = 0,
      num_train_steps = total_train_steps,
      weight_decay_rate = 0.01
  )

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

  model.compile(
      optimizer = optimizer,
      loss = loss,
      metrics = tf.metrics.SparseCategoricalAccuracy()
  )

  return model

model = create_model(num_epochs, initial_lr)

Downloading:   0%|          | 0.00/644 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at limsc/reqroberta-tapt-epoch43 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


This chunk of code allows you to control which of the encoder layers have been frozen.

In [ ]:
from transformers import models

for layer in model.layers[:]:
    print(layer, layer.trainable)

print('=========================================================================')

encoder_layer_name = {
    'bert-base-cased' : models.bert.modeling_tf_bert.TFBertMainLayer,
    'roberta-base' : models.roberta.modeling_tf_roberta.TFRobertaMainLayer,
    'allenai/scibert_scivocab_cased' : models.bert.modeling_tf_bert.TFBertMainLayer
}

frozen_layers = []

for layer in model.layers[:]:
  
  # Replace transformers.models.bert.modeling_tf_bert.TFBertMainLayer
  # with the corresponding MainLayer name from the previous code output
  if isinstance(layer, encoder_layer_name[model_type_dict[model_checkpoint]]):
    
    for idx, layer in enumerate(layer.encoder.layer):
      
      if idx in frozen_layers:
        layer.trainable = False
      
      # Confirm the chosen layers are frozen
      print(layer, layer.trainable)

<transformers.models.roberta.modeling_tf_roberta.TFRobertaMainLayer object at 0x7fb75022db50> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaClassificationHead object at 0x7fb6da21bd10> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7fb750095b10> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7fb6da350650> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7fb6da32c090> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7fb6da33ef50> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7fb6da2d7cd0> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7fb6da2eec10> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7fb6da287bd0> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7fb6da29ec90> True
<transformers.models.roberta.mode

In [ ]:
model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 124055040 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 592130    
 ficationHead)                                                   
                                                                 
Total params: 124,647,170
Trainable params: 124,647,170
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import math
import os
import numpy as np
from tensorflow.keras.callbacks import Callback, CSVLogger, ModelCheckpoint
from transformers.keras_callbacks import PushToHubCallback
from sklearn.metrics import f1_score

class macro_F1(Callback):

    def __init__(self):    
        super(macro_F1, self).__init__()

    def on_epoch_end(self, epoch, logs = {}):

        y_train_true = tokenized_ds['train']['is_functional']
        y_train_pred = np.argmax(self.model.predict(batched_train_ds)['logits'], axis = 1)
        logs['train_macro_f1'] = f1_score(y_train_true, y_train_pred)

        y_val_true = tokenized_ds['val']['is_functional']
        y_val_pred = np.argmax(self.model.predict(batched_val_ds)['logits'], axis = 1)
        logs['val_macro_f1'] = f1_score(y_val_true, y_val_pred)

        logs['seed'] = seed
        logs['batch_size'] = batch_size
        logs['learning_rate'] = initial_lr

macro_F1_cb = macro_F1()

csvlogger_file = f'{model_name_dict[model_checkpoint]}-{task_name_dict[ds_name]}.csv'
csvlogger_cb = CSVLogger(csvlogger_file, append = True)

In [ ]:
callbacks = [macro_F1_cb, csvlogger_cb]

In [ ]:
# model.fit(
#     batched_train_ds,
#     validation_data = batched_val_ds,
#     epochs = num_epochs,
#     callbacks = callbacks
# )

# Hyperparameter tuning

Perform a full grid search of all `batch_sizes` and `initial_lrs`.

In [ ]:
batch_sizes = [16, 32]
initial_lrs = [5e-5, 3e-5, 2e-5]
seeds = [21916, 25412, 56281, 61712, 30488,
         28215, 78867, 87843, 67918, 93327,
         95420, 11905, 86349, 12082, 81996]

num_epochs = 5

for batch_size in batch_sizes:

  batched_train_ds, batched_val_ds, batched_test_ds = batching(tokenized_ds, batch_size)

  for initial_lr in initial_lrs:
    
    for seed in seeds:
    
      tf.random.set_seed(seed)
      model = create_model(num_epochs, initial_lr)

      frozen_layers = []

      for layer in model.layers[:]:
        
        if isinstance(layer, encoder_layer_name[model_type_dict[model_checkpoint]]):
          
          for idx, layer in enumerate(layer.encoder.layer):
            
            if idx in frozen_layers:
              layer.trainable = False

      csvlogger_file = f'{task_name_dict[ds_name]}-{model_name_dict[model_checkpoint]}.csv'
      csvlogger_cb = CSVLogger(csvlogger_file, append = True)

      callbacks = [macro_F1_cb, csvlogger_cb]
      
      model.fit(
          batched_train_ds,
          validation_data = batched_val_ds,
          epochs = num_epochs,
          callbacks = callbacks
      )

# Model Evaluation

These are the chosen fine-tuning hyperparameters for each of the 6 models.

In [ ]:
mod_hyperparam_dict = {
    'bert-base-cased' : {'batch_size' : 16, 'initial_lr' : 2e-5, 'num_epochs' : 2},
    'roberta-base' : {'batch_size' : 32, 'initial_lr' : 3e-5, 'num_epochs' : 2},
    'allenai/scibert_scivocab_cased' : {'batch_size' : 32, 'initial_lr' : 3e-5, 'num_epochs' : 2},
    'limsc/reqbert-tapt-epoch29' : {'batch_size' : 16, 'initial_lr' : 2e-5, 'num_epochs' : 2},
    'limsc/reqroberta-tapt-epoch43' : {'batch_size' : 16, 'initial_lr' : 2e-5, 'num_epochs' : 2},
    'limsc/reqscibert-tapt-epoch20' : {'batch_size' : 32, 'initial_lr' : 5e-5, 'num_epochs' : 2}
}

In [ ]:
from sklearn.metrics import classification_report

mods = list(mod_hyperparam_dict.keys())
results_list = []
seeds = [21916, 25412, 56281, 61712, 30488,
         28215, 78867, 87843, 67918, 93327,
         95420, 11905, 86349, 12082, 81996]

for mod in mods:

    model_checkpoint = mod

    batch_size = mod_hyperparam_dict[mod]['batch_size']
    initial_lr = mod_hyperparam_dict[mod]['initial_lr']
    num_epochs = mod_hyperparam_dict[mod]['num_epochs']

    tokenizer = AutoTokenizer.from_pretrained(
        model_type_dict[mod],
        use_fast = True
    )

    tokenized_ds = ds.map(encode, batched = True)

    data_collator = DataCollatorWithPadding(
        tokenizer = tokenizer,
        return_tensors = 'tf'
    )

    batched_train_ds, batched_val_ds, batched_test_ds = batching(tokenized_ds, batch_size)

    for seed in seeds:

        tf.random.set_seed(seed)
        model = create_model(num_epochs, initial_lr)

        model.fit(
            batched_train_ds,
            epochs = num_epochs
        )

        y_true = tokenized_ds['test'][output_col]
        y_pred = np.argmax(model.predict(batched_test_ds)['logits'], axis = 1)
        
        rep = classification_report(
            y_true,
            y_pred,
            target_names = ['nfr', 'fr'],
            output_dict = True
        )

        result_dict = {
            'model': mod,
            'seed' : seed,
            'nfr_p' : format(rep['nfr']['precision'], '.4f'),
            'nfr_r' : format(rep['nfr']['recall'], '.4f'),
            'nfr_f1' : format(rep['nfr']['f1-score'], '.4f'),
            'fr_p' : format(rep['fr']['precision'], '.4f'),
            'fr_r' : format(rep['fr']['recall'], '.4f'),
            'fr_f1' : format(rep['fr']['f1-score'], '.4f'),
            'macro_p' : format(rep['macro avg']['precision'], '.4f'),
            'macro_r' : format(rep['macro avg']['recall'], '.4f'),
            'macro_f1' : format(rep['macro avg']['f1-score'], '.4f'),
            'weight_p' : format(rep['weighted avg']['precision'], '.4f'),
            'weight_r' : format(rep['weighted avg']['recall'], '.4f'),
            'weight_f1' : format(rep['weighted avg']['f1-score'], '.4f')
        }

        results_list.append(result_dict)

Loading cached processed dataset at /root/.cache/huggingface/datasets/limsc___parquet/limsc--fr-nfr-classification-665313997f6e1426/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-1184c787555408cd.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/limsc___parquet/limsc--fr-nfr-classification-665313997f6e1426/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-596ec3015d70f8a0.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/limsc___parquet/limsc--fr-nfr-classification-665313997f6e1426/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-12a61bc82a1f6301.arrow
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task 

Epoch 1/2
42/42 [==============================] - 22s 138ms/step - loss: 0.6240 - sparse_categorical_accuracy: 0.6293
Epoch 2/2
42/42 [==============================] - 6s 138ms/step - loss: 0.4014 - sparse_categorical_accuracy: 0.8445


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
42/42 [==============================] - 22s 137ms/step - loss: 0.6510 - sparse_categorical_accuracy: 0.5874
Epoch 2/2
42/42 [==============================] - 6s 137ms/step - loss: 0.4140 - sparse_categorical_accuracy: 0.8520


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch 1/2
21/21 [==============================] - 20s 246ms/step - loss: 0.4672 - sparse_categorical_accuracy: 0.7788
Epoch 2/2
21/21 [==============================] - 5s 246ms/step - loss: 0.1827 - sparse_categorical_accuracy: 0.9238


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at limsc/reqscibert-tapt-epoch20 and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
21/21 [==============================] - 20s 246ms/step - loss: 0.4930 - sparse_categorical_accuracy: 0.7474
Epoch 2/2
21/21 [==============================] - 5s 246ms/step - loss: 0.1782 - sparse_categorical_accuracy: 0.9312


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at limsc/reqscibert-tapt-epoch20 and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
21/21 [==============================] - 20s 246ms/step - loss: 0.4469 - sparse_categorical_accuracy: 0.7788
Epoch 2/2
21/21 [==============================] - 5s 246ms/step - loss: 0.1667 - sparse_categorical_accuracy: 0.9342


In [ ]:
import pandas as pd

test_df = pd.DataFrame(results_list)
test_filename = f'{task_name_dict[ds_name]}-test.csv'
test_df.to_csv(test_filename)